In [7]:
#loading necessary libraries
import nltk
nltk.download("punkt")
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import numpy as np
#from collections import defaultdict
#from nltk import pos_tag
from nltk.stem import PorterStemmer 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
#setting same randomization every time 
np.random.seed(500)


In [9]:
init_df=pd.read_csv("train main.csv" , encoding="latin-1", usecols=['category','text'] , delimiter=",")

In [10]:
#cleaning and tokenizing begins
init_df["text"].dropna(inplace=True)
init_df['text']=[entry.lower() for entry in init_df['text']]
init_df['text']= [word_tokenize(entry) for entry in init_df['text']]

In [11]:
#check if the categoris have nearly same number of texts
init_df['category'].value_counts()

category
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64

In [ ]:
lemmatizer= WordNetLemmatizer()
Stemmer = PorterStemmer()
for index, entry in enumerate(init_df['text']) :
    Final_words = []
    for word in entry :
           if word not in stopwords.words('english') and word.isalpha() :
            word_Final = Stemmer.stem(word)
            Final_words.append(word_Final)
           elif not word.isalpha(): 
                Final_words.append(word)
    init_df.loc[index,'cleaned_text']=str(Final_words)    
    

C:\Users\USER\AppData\Local\Temp\ipykernel_14032\2635318096.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['tv', 'futur', 'hand', 'viewer', 'home', 'theatr', 'system', 'plasma', 'high-definition', 'tv', 'digit', 'video', 'record', 'move', 'live', 'room', 'way', 'peopl', 'watch', 'tv', 'radic', 'differ', 'five', 'year', 'time', '.', 'accord', 'expert', 'panel', 'gather', 'annual', 'consum', 'electron', 'show', 'la', 'vega', 'discuss', 'new', 'technolog', 'impact', 'one', 'favourit', 'pastim', '.', 'us', 'lead', 'trend', 'programm', 'content', 'deliv', 'viewer', 'via', 'home', 'network', 'cabl', 'satellit', 'telecom', 'compani', 'broadband', 'servic', 'provid', 'front', 'room', 'portabl', 'devic', '.', 'one', 'talked-about', 'technolog', 'ce', 'digit', 'person', 'video', 'record', '(', 'dvr', 'pvr', ')', '.', 'set-top', 'box', 'like', 'us', 'tivo', 'uk', 'sky+', 'system', 'allow', 'peopl', 'record', 'store', '

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(init_df['cleaned_text'],init_df['category'],test_size=0.2)

In [ ]:
#check if the train dataset's categoris have nearly same number of texts
min=Train_Y.value_counts().min()

In [ ]:
Tfidf_vect = TfidfVectorizer(min_df=4)
Tfidf_vect.fit(init_df['cleaned_text'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
NB = naive_bayes.MultinomialNB(alpha=1)
NB.fit(Train_X_Tfidf,Train_Y)

# predict the labels on the test dataset
pred_NB = NB.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(pred_NB, Test_Y)*100)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
confmatrix=confusion_matrix(pred_NB, Test_Y)
print (confmatrix)

In [ ]:
columns = Tfidf_vect.get_feature_names_out()
train_tfidf_df = pd.DataFrame(Train_X_Tfidf.toarray(), columns=columns)
test_tfidf_df = pd.DataFrame(Test_X_Tfidf.toarray(), columns=columns)